In [5]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from datetime import datetime, timedelta
from pathlib import Path

In [6]:
colors = ["#1f77b4", "#1f77b4", "#ff7f0e", "#ff7f0e", "tab:olive", "tab:olive", "tab:olive"]

In [8]:
pipeline_steps = {}

pipeline_steps['Image VAE-Encoder'] = pd.read_csv(Path("../inference_tests/image_vae/encoder.csv"),
                          index_col=0,
                          dtype={'frameID': int, 'inference_time': float})
pipeline_steps['Image VAE-Decoder'] = pd.read_csv(Path("../inference_tests/image_vae/decoder.csv"),
                          index_col=0,
                          dtype={'frameID': int, 'inference_time': float})
pipeline_steps['Image GAN-Encoder'] = pd.read_csv(Path("../inference_tests/image_gan/encoder.csv"),
                          index_col=0,
                          dtype={'frameID': int, 'inference_time': float})
pipeline_steps['Image GAN-Decoder'] = pd.read_csv(Path("../inference_tests/image_gan/decoder.csv"),
                          index_col=0,
                          dtype={'frameID': str, 'inference_time': float}) 
                          # Attention with frameID here, its wrong in the source data
pipeline_steps['Lidar-Encoder'] = pd.read_csv(Path("../inference_tests/lidar/encoder.csv"),
                          index_col=0,
                          dtype={'frameID': int, 'inference_time': float})
pipeline_steps['Lidar-Decoder'] = pd.read_csv(Path("../inference_tests/lidar/decoder.csv"),
                          index_col=0,
                          dtype={'frameID': int, 'inference_time': float})  
pipeline_steps['Lidar-Pre-Processor'] = pd.read_csv(Path("../inference_tests/lidar/preprocessor.csv"),
                          index_col=0,
                          dtype={'frameID': int, 'inference_time': float})

pipeline_steps['Image VAE-Encoder'].rename(columns = {'inference_time_compression':'inference_time_process'}, inplace = True)
pipeline_steps['Image VAE-Decoder'].rename(columns = {'inference_time_decompress':'inference_time_process'}, inplace = True)
pipeline_steps['Image GAN-Encoder'].rename(columns = {'inference_time_compression':'inference_time_process'}, inplace = True)
pipeline_steps['Image GAN-Decoder'].rename(columns = {'inference_time_decompress':'inference_time_process'}, inplace = True)
pipeline_steps['Lidar-Encoder'].rename(columns = {'inference_time_compression':'inference_time_process'}, inplace = True)
pipeline_steps['Lidar-Decoder'].rename(columns = {'inference_time_decompress':'inference_time_process'}, inplace = True)
pipeline_steps['Lidar-Pre-Processor'].rename(columns = {'inference_time_preprocess':'inference_time_process'}, inplace = True)



for processing_step in pipeline_steps:
    pipeline_steps[processing_step]['timestamp'] = pd.to_datetime(pipeline_steps[processing_step]['timestamp'])
    pipeline_steps[processing_step]['elapsed_time'] = pipeline_steps[processing_step]['timestamp'] - pipeline_steps[processing_step]['timestamp'].iloc[0]
    pipeline_steps[processing_step] = pipeline_steps[processing_step][pipeline_steps[processing_step]['elapsed_time'] <= timedelta(minutes=5, seconds=0, milliseconds=0)]
    print(processing_step)
    # print(pipeline_steps[processing_step].dtypes)
    print(pipeline_steps[processing_step]['elapsed_time'].max())


Image VAE-Encoder
0 days 00:04:59.993878
Image VAE-Decoder
0 days 00:04:59.977382
Image GAN-Encoder
0 days 00:04:59.866946
Image GAN-Decoder
0 days 00:04:59.860091
Lidar-Encoder
0 days 00:04:59.914534
Lidar-Decoder
0 days 00:04:59.883778
Lidar-Pre-Processor
0 days 00:04:59.922256


In [9]:
data = []
for processing_step in pipeline_steps:
    num_images = pipeline_steps[processing_step]['frameID'].count()
    images_per_second = num_images / (5*60)
    avg_inference_time = pipeline_steps[processing_step]['inference_time'].mean()
    avg_inference_time_process = pipeline_steps[processing_step]['inference_time_process'].mean()
    data.append({'Processing Step': processing_step,
                 'Number of Processed Images': num_images,
                 'Throughput in [images/s]': images_per_second,
                 'Average Inference Time in [ms]': avg_inference_time,
                 'Average Inference Time Process in [ms]': avg_inference_time_process})
results = pd.DataFrame(data)
print(results)

       Processing Step  Number of Processed Images  Throughput in [images/s]  \
0    Image VAE-Encoder                        2890                  9.633333   
1    Image VAE-Decoder                        2890                  9.633333   
2    Image GAN-Encoder                        1845                  6.150000   
3    Image GAN-Decoder                        1241                  4.136667   
4        Lidar-Encoder                        1867                  6.223333   
5        Lidar-Decoder                        1867                  6.223333   
6  Lidar-Pre-Processor                        1867                  6.223333   

   Average Inference Time in [ms]  Average Inference Time Process in [ms]  
0                       34.894084                               16.876568  
1                       29.139152                               20.544476  
2                      102.384505                               93.426444  
3                      237.630653                      

In [10]:
results

,Processing Step,Number of Processed Images,Throughput in [images/s],Average Inference Time in [ms],Average Inference Time Process in [ms]
0,Image VAE-Encoder,2890,9.633333,34.894084,16.876568
1,Image VAE-Decoder,2890,9.633333,29.139152,20.544476
2,Image GAN-Encoder,1845,6.150000,102.384505,93.426444
3,Image GAN-Decoder,1241,4.136667,237.630653,230.722214
4,Lidar-Encoder,1867,6.223333,23.889529,10.350815
5,Lidar-Decoder,1867,6.223333,37.600162,8.914099
6,Lidar-Pre-Processor,1867,6.223333,146.242212,114.215640


In [11]:
sns.set_style("whitegrid")

matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
    'axes.labelsize': 12,
    'font.size': 12,
})


f, ax = plt.subplots()

f.set_size_inches(w=5.50107, h=3) # latex textwidth=5.50107in and we have 2 columns
bar_total = sns.barplot(x="Average Inference Time in [ms]",
            y="Processing Step",
            data=results,
            label="ROS",
            color="lightgrey")

bar_models = sns.barplot(x="Average Inference Time Process in [ms]",
            y="Processing Step",
            data=results,
            # label="Model",
            palette=colors)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels)

ax.set_xticks(range(0,350,50))
f.tight_layout()

plt.savefig('online-bar-inference-with-ROS-overhead.pgf')

In [13]:
results_adj = results.copy()

values = ["Image VAE-Encoder", "Image GAN-Encoder", "Lidar-Encoder", "Lidar-Pre-Processor"]


results_adj = results_adj[results_adj["Processing Step"].isin(values) == False]
results_adj = results_adj.reset_index(drop=True)

results_adj.at[0, 'Processing Step'] = "Image VAE"
results_adj.at[1, 'Processing Step'] = "Image GAN"
results_adj.at[2, 'Processing Step'] = "Lidar"


results_adj

,Processing Step,Number of Processed Images,Throughput in [images/s],Average Inference Time in [ms],Average Inference Time Process in [ms]
0,Image VAE,2890,9.633333,29.139152,20.544476
1,Image GAN,1241,4.136667,237.630653,230.722214
2,Lidar,1867,6.223333,37.600162,8.914099


In [14]:
colors_adj = ["#1f77b4", "#ff7f0e", "tab:olive"]

In [15]:
sns.set_style("whitegrid")

matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
    'axes.labelsize': 12,
    'font.size': 12,
})


f, ax = plt.subplots()

f.set_size_inches(w=5.50107, h=2) # latex textwidth=5.50107in and we have 2 columns
sns.barplot(x="Throughput in [images/s]",
            y="Processing Step",
            data=results_adj,
            label="Total",
            palette=colors_adj
           )
ax.set_xticks(range(11))

f.tight_layout()

plt.savefig('online-bar-tp-img.pgf')